This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules

import requests
import json

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
data_sample = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2014-12-01&end_date=2014-12-01&api_key=')

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(data_sample.json())

{'dataset': {'id': 10095370, 'dataset_code': 'AFX_X', 'database_code': 'FSE', 'name': 'Carl Zeiss Meditec (AFX_X)', 'description': 'Stock Prices for Carl Zeiss Meditec (2020-08-31) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704', 'refreshed_at': '2020-11-20T22:16:29.120Z', 'newest_available_date': '2020-11-06', 'oldest_available_date': '2000-06-07', 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'frequency': 'daily', 'type': 'Time Series', 'premium': False, 'limit': None, 'transform': None, 'column_index': None, 'start_date': '2014-12-01', 'end_date': '2014-12-01', 'data': [['2014-12-01', 22.1, 22.14, 21.82, 21.91, None, 21706.0, 477060.0, None, None, None]], 'collapse': None, 'order': None, 'database_id': 6129}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
#1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
data = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&api_key=')

In [6]:
#2. Convert the returned JSON object into a Python dictionary.

AFX_dict = data.json()
type(AFX_dict)


dict

In [7]:
#3. Calculate what the highest and lowest opening prices were for the stock in this period.
open_price = []

for data in AFX_dict['dataset']['data']: 
    if type(data[1]) == float:
        open_price.append(data[1])

high_open = max(open_price)
low_open = min(open_price)
print('The highest opening price was: $' + str(high_open))
print('The lowest opening price was: $' + str(low_open))

The highest opening price was: $53.11
The lowest opening price was: $34.0


In [8]:
#4. What was the largest change in any one day (based on High and Low price)?
price_changes = []

for data in AFX_dict['dataset']['data']: 
    if type(data[2]) == float and type(data[3]) == float:
        change = data[2] - data[3]
        price_changes.append(change)

print('The largest change in any one day: $' + str(max(price_changes)))

The largest change in any one day: $2.8100000000000023


In [9]:
#5. What was the largest change between any two days (based on Closing Price)?
dates = []
opens = []
closes = []
date_open_close = []

for i, data in enumerate(reversed(AFX_dict['dataset']['data'])): 
    #gotta make a separate date, open, close array, and then do the i and i-1 analysis
    
    if type(data[1]) == float and type(data[4]) == float:
        dates.append(data[0])
        opens.append(data[1])
        closes.append(data[4])

date_open_close = [dates, opens, closes]

daily_changes = []

for i in range(len(dates)):
    if i != 0:
        d_change = opens[i] - closes[i-1]
        daily_changes.append(d_change)
        
print('The largest change between any two days was: $' + str(max(daily_changes)))

The largest change between any two days was: $0.7800000000000011


In [11]:
#6. What was the average daily trading volume during this year?
trading_volumes = []

for data in AFX_dict['dataset']['data']: 
    if type(data[6]) == float or type(data[6]) == int:
        trading_volumes.append(data[6])
        
avg = sum(trading_volumes) / len(trading_volumes)
print('The average daily trading volume during this year (trading_volumes): ' + str(avg))

The average daily trading volume during this year (trading_volumes): 89124.33725490196


In [12]:
#7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)
s_tv = sorted(trading_volumes)
half = len(s_tv)/2 
median = s_tv[int(half)]
print('The median for the trading volume is: ' + str(median))

The median for the trading volume is: 76286.0
